In [ ]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.pipeline_util as pu
import utils.transformer_util as tu
from utils.prediction_builder import PredictionDFBuilder
import numpy as np
import IMDb_Predictor_GUI_II as gui
import tkinter as tk
from tkinter import ttk

In [ ]:
%gui tk

In [ ]:
# set pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

In [ ]:
# run the pipeline and get the model and predictions
model, best_y_pred, preprocessed_df = pu.run_pipeline(data=df, use_PCA=False, debug=False)

In [ ]:
# describe the predictions
pd.Series(best_y_pred).describe()

In [ ]:
# create a prediction dataframe for testing
builder = PredictionDFBuilder(df)
prediction_df = (
    builder
    .add_actor_1("Clint Eastwood")
    .add_actor_2("Meryl Streep")
    .add_actor_3("Tom Hanks")
    .add_director("Gore Verbinski")
    .add_rating("PG-13")
    .add_genre("Thriller")
    .build()
)

prediction_df


In [ ]:
# predict the test data without GUI
prediction = model.predict(prediction_df)
print(f"Predicted IMDb Score: {prediction[0]}")

In [ ]:
# run the GUI to get user input and display the prediction
root = tk.Tk()
gui = gui.PredictionDFBuilderGUI(root, df, model)
root.mainloop()
